<a href="https://colab.research.google.com/github/schwallergroup/ai4chem_course/blob/scikit_learn/notebooks/02%20-%20Supervised%20Learning/training_and_evaluating_ml_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Run this cell to install everything you need for this tutorial.

!pip install numpy

# TODO: Set up the environment. This at the end when we know what libraries we need, etc
# TODO: Download all the files we need

# Week 2 tutorial - AI 4 Chemistry

## Index:

- Classification
.
.
.

### TODO

# Introduction to traditional ML.

## Supervised learning

Training a model to take inputs X and return output y.

As you have seen in class, for this type of learning, we have two variants:

- Classification
- Regression

Linear regression is one example of suppervised learning for regression.

# Regression

### TODO: Improve this introduction based on the ESOL paper, why is solub. prediction important?

One problem in both academic and industrial chemistry is predicting solubility. For instance we might know that some molecule has good potential as a ligand for some relevant reaction, however when you synthesize it, you realize it's not soluble under your already optimized reaction conditions! 😥

It would be extremely useful to know the solubility of my molecule, **before I even try to synthesize it**!

---

In this task we will try to solve this using supervised learning. In particular, we will train a regression model using the very convenient [scikit-learn](https://www.kaggle.com/competitions/MerckActivity/data) Python library, to predict solubility based on some molecular descriptors.

In [13]:
# TODO: Let's start with loading the data, visualizing some molecules and their solubility
# Let's also see some stats. e.g. size of dataset, distribution of solubility, etc.

# TODO: Generate features
# TODO do a train/test split

In [3]:
# TODO: Let's give an introduction to scikit learn by doing a simple linear regression and see the results.
# TODO Introduce sckit-learn, and use a RF model for this.
import tempfile
from sklearn.manifold import Isomap
from sklearn.neighbors import KNeighborsTransformer
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_regression
cache_path = tempfile.gettempdir()  # we use a temporary folder here
X, _ = make_regression(n_samples=50, n_features=25, random_state=0)
estimator = make_pipeline(
    KNeighborsTransformer(mode='distance'),
    Isomap(n_components=3, metric='precomputed'),
    memory=cache_path)
X_embedded = estimator.fit_transform(X)
X_embedded.shape

In [16]:
# TODO: Let's train another model (this can be an excercise)

# EXERCISE: Implement random forest and and XGBoost models using scikit learn

# As we see, there are many possible models we can use for this task. But which one is better?

In addition, each model has a set of hyperparameters that we need to tune ourselves. How do we select them?

This is an important part of machine learning! What we want to know is: What is the best combination of model + model hyperparameters for our task? 
As you've seen in the course, common strategies to evaluate and compare model's performance include:

- Splitting dataset in train/validation/test.
- Doing cross-validation for hyperparameter tuning.

In [7]:
# TODO: Split data in train/valid/test

# Retrain the models on the train set, and compare them using the validation set.

# What model is the best?

In [14]:
# TODO: Let's do cross-validation

# Optimize the hyperparameters for XGBoost, and again compare performance

In [15]:
# TODO: Finally, compare all models on the test set.
# Explain that test set should never be seen by models.
# This is all completely new data so we know how it would work in real life.

---

# Classification

We now turn our attention towards the other type of supervised learning: classification.

Many questions in chemistry can be framed as a classification task: 

- Will this molecule act as a nucleophile or electrophile in my reaction?
- What is the smell of this substance? (fruity, citrus, sweet, ...)

<div>
<img src="img/is_this_a_meme.png" width="500"/>
</div>


--- 

### TODO: Let's get a dataset for classification in molecules. Let's say it's prediction of toxicity.

What we want to know is, is the molecule shown toxic or not?

##### TODO insert image of molec

Let's see if a model can tell what molecules are toxic!

This would be very useful for instance in drug discovery, where we want to know if a molecule has potential as a drug, **even before we synthesize it**.

## To do this, we will use [mordred](http://mordred-descriptor.github.io/documentation/master/descriptors.html) to generate some molecular descriptors, and will again train some models using scikit-learn using these features.


In [16]:
#!gzip -dk data/clintox.csv.gz
!pip install mordred
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.3/29.3 MB 28.8 MB/s eta 0:00:0000:0100:01


In [32]:
!pip uninstall mordred
!conda install -c rdkit -c mordred-descriptor mordred

Found existing installation: mordred 1.2.0
Uninstalling mordred-1.2.0:
  Would remove:
    /home/andres/anaconda3/lib/python3.9/site-packages/mordred-1.2.0.dist-info/*
    /home/andres/anaconda3/lib/python3.9/site-packages/mordred/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
Retrieving notices: ...working... done
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - ^C
failed

CondaError: KeyboardInterrupt



In [17]:
import pandas as pd

df_toxicity = pd.read_csv("data/clintox.csv")
print(df_toxicity.head(3))

df_toxicity["CT_TOX"].value_counts()

                                              smiles  FDA_APPROVED  CT_TOX
0            *C(=O)[C@H](CCCCNC(=O)OCCOC)NC(=O)OCCOC             1       0
1  [C@@H]1([C@@H]([C@@H]([C@H]([C@@H]([C@@H]1Cl)C...             1       0
2  [C@H]([C@@H]([C@@H](C(=O)[O-])O)O)([C@H](C(=O)...             1       0


0    1372
1     112
Name: CT_TOX, dtype: int64

In [18]:
from rdkit import Chem
from mordred import Calculator, descriptors

# create descriptor calculator with all descriptors
calc = Calculator(descriptors, ignore_3D=True)

# calculate single molecule
mol = Chem.MolFromSmiles('c1ccccc1')
calc(mol)[:3]

# calculate multiple molecule
mols = [Chem.MolFromSmiles(smi) for smi in ['c1ccccc1Cl', 'c1ccccc1O', 'c1ccccc1N']]

# as pandas
df = calc.pandas(mols)
df['SLogP']


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 51.75it/s]


0    2.3400
1    1.3922
2    1.2688
Name: SLogP, dtype: float64

In [28]:
mols = df_toxicity.smiles.apply(lambda x: Chem.MolFromSmiles(x, sanitize=False))

In [31]:
calc.pandas(mols)

  0%|                                                                                                                        | 0/1484 [00:00<?, ?it/s][21:00:38] 

****
Pre-condition Violation
getNumImplicitHs() called without preceding call to calcImplicitValence()
Violation occurred on line 297 in file /project/build/temp.linux-x86_64-cpython-39/rdkit/Code/GraphMol/Atom.cpp
Failed Expression: d_implicitValence > -1
****

  0%|                                                                                                                        | 0/1484 [00:00<?, ?it/s]


RuntimeError: Pre-condition Violation
	getNumImplicitHs() called without preceding call to calcImplicitValence()
	Violation occurred on line 297 in file Code/GraphMol/Atom.cpp
	Failed Expression: d_implicitValence > -1
	RDKIT: 2022.09.4
	BOOST: 1_78


In [ ]:
# TODO load dataset
# TODO small visualization of dataset, let's see some molecules and the property we want to predict.

In [ ]:
# TODO train a Random Forest classification model

In [ ]:
# TODO evaluate model using metrics for this (AUC-ROC, accuracy, etc)

In [ ]:
# TODO explore feature importance
# Do these features make sense?
# Find out in moldred documentation what the features are and think why this is important for the model.

In [11]:
# TODO train another model (maybe XGBoost) and explain using SHAP

# Tasks for today: 

- scikit learn
- classification
- regression: [ESOL dataset](https://www.kaggle.com/competitions/MerckActivity/data)
- XGBoost: [Merck kaggle challenge](https://www.kaggle.com/competitions/MerckActivity/data)
- SHAP values + feature importance
- Evaluation of ML models
- Cross-validation: hyperparameter tuning
- Train/valid/test split